## **Building a Minimal Parsing Library Using Parser Combinators in Haskell**

A *parser* is just a translation function: a functor/function that takes an input from one category (of some data structure) to another. It takes in loosely-structured data and translates it to another structured data, based on some grammar.

Therefore, a parser should have the following general signature:
```haskell 
data Parser'''' m a n b = Parser'''' m a -> n b
```
This one is, of course, *very* general. It takes an argument of type $m \; a$ (where $m$ is a type constructor) and translates it to another one of type $n \; b$ (where n is also another type constructor), where $m$ and $n$ are both functors/type constructors. However, since parsers work sequentially—that is, they parse part of the input stream, then output the translated value and the rest of the stream. Therefore, we need to know where the parser is or what is left to process. It also may not output something if the input streams doesn't follow the grammar to be processed!

```haskell
data Parser''' m a n b = Parser''' (m a -> Either ParserError (n b, m a)) 
--`n b` in the pair is the what is successfully "parsed out" or translated, m a is what is left of the stream to process.
```

Now, let's say we want our parser to process only a stream of characters—that is, a String/list of Char. In that case, the type signature of our parser should be as follows:
```haskell
data Parser'' n b = Parser'' ([Char] -> Either ParserError (n b, [Char]))
```
which can be shortened to:

```haskell
data Parser' output = Parser' ([Char] -> Either ParserError (output, [Char]))
-- `output` is the type of what the parser "parses out".
```

A nicer way to declare it's type is using the record syntax in Haskell:
<!-- ```haskell
data Parser output = Parser' ([Char] -> Either PraserError (output, [Char]))``` -->


In [ ]:
import           Control.Applicative (liftA2)
import           Data.Char
import           Data.Foldable       (for_)
import           Data.Functor
import qualified Data.HashMap.Strict as M
import           Data.List           (intercalate)
import           Prelude             hiding (any)
import           System.Environment
import           Text.Printf
-- Definition of the parser output when it can't parse the input stream.
data ParserError = ParserError {getExpected :: String, getFound :: String} deriving Show

newtype Parser output = Parser {runParser :: [Char] -> Either (ParserError, [Char]) (output, [Char])}

Now, let's say we want to create a parser that parses for the End of File character. It can be done as follows:

In [ ]:
eofParser :: Parser [Char] -- Used [Char] here instea
eofParser = Parser (\input -> case input of
                    []       -> Right ("", [])
                    (char:_) -> Left (ParserError "Expected: EoF" ("Found: " ++ [char]), input) 
    )

(runParser eofParser "")

We are also going to define another parser that parses for any character (this will come in handy later):

In [ ]:
any :: Parser Char
any = Parser $ \input ->
                case input of
                (char:chars) -> Right (char, chars)
                _ -> Left (ParserError "Expected: Char" "Found: EoF", input) 

-- generalAny :: Parser a
-- generalAny = Parser $ \input -> 
--                        case input of
--                             (x:xs) -> Right (x, xs)
--                             _ -> Left $ ParserError "Expected: Char" "Found: EOF"

$\text{errorParser}$ is a function produces a special parser that always parsers out a $ParserError$ regardless of the input. This will prove useful later one when we "choose" between different parsers and no one of them parser the input successfully.

In [ ]:
errorParser :: String -> String -> Parser a
errorParser expected found = Parser $ \input -> Left (ParserError expected found, input)

#### **How About Sequencing Parsers?**

So, now, what does sequencing parsers actually mean? Well, it simply means running a parser on the rest of the input stream out of another parser *after* it parses successfully. And since a parser of type `Parser a` spits out values of type `Either ParserError (parserOutput :: a, restOfInput :: String)`, we are only interested in passing `restOfInput` to the other parser `Parser b`, which may parses out values of potentially different type `b`, only *when* the value of the output is `Right (parserOutput :: a, restOfInput::String)`. So, in fewer words, what we want to do is run `Parser a` on `input :: String`, check whether its output is a `Right (parserOutput :: a, restOfInput :: String)` value. Ff true, we run `Parser B` on `restOfInput`. If false, we may throw an error. 
Here is a psuedo type-signature for the what we are trying to implement here to make things clearer:
```haskell
parseThenDo :: Parser a -> (a -> Parser b) -> Parser b 
-- while `a` is not exactly the type of the output of teh function wrapped in `Parser a', both `Parser a` and `Parser b` operate on and produces the same `input, restOfInput :: String` values. ^[I need a better explanation here to what is actually going on.] 
```
Let's try to interpret this type signature: `parseThenDo` takes a `parserA :: Parser a` and another function that takes in some input and produces another `parserB` of potentially different type `Parser b`. And now compare this type signature with the that of the bind function `(>>=)` of monads in Haskell:
```haskell
(>>=) :: m a -> (a -> m b) -> m b 
``` 
This is specifically why monads are so darn useful: `>>=` takes a value of type `m a` and another functions that does some computation on the values of type `a` inside the monad container `m a` to produce another monad containers of potentially different type `m b`.

That way, by implementing the Monad typeclass on`Parser`, we can overload the bind function `>>=` to do the parser sequencing for us by taking in a `parserA` and a function `f` that handles the logic of piping the output of `parserA` to another `parserB :: Parser c` to produce another `parserC :: Parser b`. As follows: 


In [ ]:
instance Functor Parser where
    fmap f parserA = Parser (\input ->
                             case runParser parserA input of
                             Right (output, restOfInput) -> Right (f output, restOfInput)
                             Left a -> Left a
                             )

instance Applicative Parser where
    pure x = Parser (\input -> Right (x, input))
    parserF <*> parserA = Parser (\input -> 
                                   case runParser parserF input of
                                   Right (f, restOfInput) -> runParser (fmap f parserA) restOfInput
                                   Left a                 -> Left a
                                 )

instance Monad Parser where
  return x = Parser $ \input -> Right (x, input)
  parserA >>= f = Parser $ \input -> case runParser parserA input of
                                       Right (output, restOfInput) -> runParser (f output) restOfInput
                                       Left a -> Left a

*$\text{generalSatisfy} :: Eq \; => \; a \to ( a \to Bool) \to Parser\;a $* is a combinator that wraps $\:\text{any}\, :: \, Parser\; a$ (where $a$ usually has the type $Char$) value of type $a$ another a predicate/function of type $Char \to Bool$ (which checks if the parsed out value (usually of type $Char$) is equal to that argument) returns a parser that parses only for that value/character. This is the first ***Parser Combinator***!

In [ ]:
-- Comment:
-- This didn't work because I need  a general Parser first
-- generalSatisfy :: Eq a => a -> (a -> Bool) -> Parser a
-- satisfy value predicate = Parser \input ->
--                                   case runParser generalAny input of
--                                        Right (x:xs) -> if x == value then Right (x:xs) 
--                                                        else Left $ ParserError ("Expected: " ++ value) ("Found: " ++ x) 
--                                        _ -> _
                                       

Now, we can makr another parser combinator that checks for a specific value of type $Char$.

In [ ]:
satisfyParser :: (Char -> Bool) -> Parser Char
satisfyParser predicate = Parser $ \input ->
                                  case runParser any input of
                                       Right (char, restOfInput) -> if predicate char then Right (char, restOfInput) 
                                                       else Left $ (ParserError "Expected: Char of certain property." ("Found: " ++[char]), input) 
                                       Left a -> Left a
-- runParser $ satisfy (== 'a') "abs"

In [ ]:
try :: Parser a -> Parser a
try parserA= Parser $ \input -> 
                       case runParser parserA input of
                            Right a -> Right a
                            Left (error, output) -> Left (error, input)

The $(<|>)$ operator combines two parsers and produces another parser. This new parser tries the first parser, and if it fails (without consuming any input), then it runs the other parser.

In [ ]:
-- (<|>) :: Parser a -> Parser b -> Parser (Either a b)
(<|>) :: Parser a -> Parser a -> Parser a
parser1 <|> parser2 = Parser $ \input ->
                                case runParser parser1 input of
                                     Right a -> Right a
                                     Left (error, input) ->  runParser parser2 input 

Given a list of parsers, $\text{chooseParser}$ tries them in order, and outputs the first parser that parses successfully.

In [ ]:
chooseParser :: [Parser a] -> Parser a 
chooseParser [parser] = parser
chooseParser (firstParser:rest) = firstParser <|> chooseParser rest

$\text{choice}$ produces a parser that choose between a list of parsers (where all parsers are tried in the same order in the list), and if they all fail, it produces a $ParserError$ with the expected field in it is the $description$ parameter. The namesake of the function is the same as the one in $\text{Parsec}$

In [ ]:
choice :: String -> [Parser a] -> Parser a
choice description parsers = foldr (<|>) (errorParser description "No match") parsers

choice' :: String -> [Parser a] -> Parser a
choice' description parsers = chooseParser $ parsers ++ [errorParser description "No match."]


$\text{many}$ is another parser combinator that parses zero or more occurence of a given value.

In [ ]:
many, many1 :: Parser a -> Parser [a]
many  p = many1 p <|> return []
many1 p = do
  first <- p
  rest  <- many p
  return (first:rest)

In [ ]:
sepBy, sepBy1 :: Parser a -> Parser s -> Parser [a]
sepBy  p s = sepBy1 p s <|> pure []
sepBy1 p s = liftA2 (:) p $ many (s >> p)

### **Using the Library: Implementing a Minimal JSON Parser**

##### **The Extended Backus-Naur Form for JSON**

VAlUE ::= STRINGLIT | NUMBER | BOOL | OBJECT | ARRAY </br> </br>
OBJECT ::= "{" [PAIR {["," PAIR]}] "} </br> </br>
PAIR ::= STRINGLIT ":" VALUE </br> </br>
ARRAY ::= "[" [VALUE [{"," VALUE}]] "]" </br> </br>

In [ ]:
import Data.Map.Strict as Map
data JValue = JString String 
            | JDouble Double 
            | JObject (Map String JValue)
            | JArray [JValue]
            | JBool Bool
            | Null
            deriving Show

In [ ]:
import Data.Char
charParser c = satisfyParser (==c)
spaceParser = satisfyParser isSpace
digitParser = satisfyParser isDigit
symbolParser symbol = charParser symbol <* spaceParser

stringParser = traverse charParser

betweenParser openParser closeParser valueParser = openParser *> valueParser <* closeParser
betweenBracketsParser = betweenParser (symbolParser '[') (symbolParser ']')
betweenBracesParser = betweenParser (symbolParser '{') (symbolParser '}')


In [ ]:
jsonNumberParser = read <$> many1 digitParser

In [ ]:
jsonBoolParser = choice ("Couldn't parse Boolean value.") [True <$ stringParser "true", False <$ stringParser "false"]

In [ ]:
jsonStringParser = choice "Couldn't parse String value." [betweenParser (symbolParser '\"') (symbolParser '\"') $ many jsonCharParser]
                                                          where 
                                                            jsonCharParser = choice "Couldn't parse a Character value."
                                                                  [ try $ '\n' <$ stringParser "\\n"
                                                                  , try $ '\t' <$ stringParser "\\t"
                                                                  , try $ '"'  <$ stringParser "\\\""
                                                                  , try $ '\\' <$ stringParser "\\\\"
                                                                  , satisfyParser (/= '"')
                                                                  ]
-- jsonStringParser = choice "Couldn't parse String value." [betweenParser (symbolParser '"') (symbolParser '"') many jsonCharParser,
--                                                           betweenParser (symbolParser '\'') (symbolParser '\'') many jsonCharParser]
--                                                           where 
--                                                             jsonCharParser = choice "Couldn't parse a Character value."
--                                                                   [ try $ '\n' <$ stringParser "\\n"
--                                                                   , try $ '\t' <$ stringParser "\\t"
--                                                                   , try $ '"'  <$ stringParser "\\\""
--                                                                   , try $ '\\' <$ stringParser "\\\\"
--                                                                   , satisfyParser (/= '"') "not a quote"
--                                                                   ]

In [ ]:
jsonObjectParser = do
  assocList <- betweenBracesParser $ jsonEntryParser `sepBy` symbolParser ','
  return $ fromList assocList
  where
    jsonEntryParser = do
      k <- jsonStringParser
      symbolParser ':'
      v <- jsonValueParser
      return (k,v)

  
jsonArrayParser = betweenBracketsParser $ jsonValueParser `sepBy` symbolParser ','

jsonValueParser = choice "Couldn't Parse JSON."  [JObject <$> jsonObjectParser, JArray  <$> jsonArrayParser, JString <$> jsonStringParser, JDouble <$> jsonNumberParser, JBool <$> jsonBoolParser , Null <$  stringParser "null"]

In [ ]:
runParser jsonObjectParser "{ \"string\": \"b\", \"string-with-escaped-stuff\": \"\"\n\t\", \"number\": 4234746,\"array\": [\"foo\", 42, {\"foo\": 42}, [\"foo\", 42]],\"bool\": true,\"other-bool\": false, \"null\": null }"

In [ ]:
main :: IO ()
main = do
  content <- readFile "test.json"
  putStrLn content
  print $ runParser jsonObjectParser content

main